In [11]:
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, 
                            height_shift_range=0.2, rescale=1./255,
                            shear_range=0.2, zoom_range=0.2, 
                             horizontal_flip=True, fill_mode='nearest')

In [3]:
img = load_img("./dogs-vs-cats/train/dog.8011.jpg") # reading with PIL
x = img_to_array(img) #Converting PIL Image to numpy array
x = x.reshape((1,) + x.shape) #Appending a new dim to the first axes

In [4]:
i = 0
for batch in datagen.flow(x, batch_size=1, seed=1, save_to_dir="preview", 
                         save_prefix="dog", save_format="jpeg"):
    i +=1
    if i == 20:
        break

In [5]:
# Thats a nice way of stating it
# Dropout also helps reduce overfitting, by preventing a layer from seeing twice the exact same pattern, thus acting in a way analoguous to data augmentation (you could say that both dropout and data augmentation tend to disrupt random correlations occuring in your data).

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [8]:
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
# Setting up hyper parameters
batch_size = 32
learning_rate = 0.001

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(
        rescale=1./255
)

train_generator = train_datagen.flow_from_directory("./dogs-vs-cats/train_1000/", target_size=(150, 150),
                                                   batch_size=batch_size,
                                                   class_mode="binary")

test_generator = train_datagen.flow_from_directory("./dogs-vs-cats/test_250/", target_size=(150, 150),
                                                   batch_size=1,
                                                   class_mode="binary")


Found 2298 images belonging to 2 classes.
Found 202 images belonging to 2 classes.


In [10]:
steps_per_epoch = len(train_generator.filenames)//train_generator.batch_size
v_steps_per_epoch = len(test_generator.filenames)//test_generator.batch_size

In [26]:
# Training/Fitting over the egnerator
model.fit_generator(train_generator, 
                    v_steps_per_epoch, 
                    epochs = 1, 
                    validation_data=test_generator)

Epoch 1/1
202/202 [==============================] - 141s 696ms/step - loss: 0.6716 - accuracy: 0.5877 - val_loss: 0.6469 - val_accuracy: 0.6337


In [11]:
model.save_weights('first_try.h5')

In [15]:
from keras.applications import VGG16

553467904/553467096 [==============================] - 196s 0us/step


In [43]:
vgg = VGG16(include_top=False, weights='imagenet')

In [44]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [49]:
batch_size = 16

datagen = ImageDataGenerator(rescale=1./255)
generator = datagen.flow_from_directory(
        './dogs-vs-cats/train_1000/',
        target_size=(50, 50),
        batch_size=batch_size,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False, )  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
        './dogs-vs-cats/test_250/',
        target_size=(50, 50),
        batch_size=8,
        class_mode=None,
        shuffle=False
)

Found 2298 images belonging to 2 classes.
Found 202 images belonging to 2 classes.


In [50]:
bottleneck_features_train = vgg.predict_generator(generator, 2298//batch_size)

In [51]:
np.save(open("bottleneck_features_train.npy", "wb"), bottleneck_features_train)

In [58]:
train_data = np.load(open('bottleneck_features_train.npy', "rb"))
train_labels = np.array([0] * 1149 + [1]* 1139)

In [59]:
train_data.shape

(2288, 1, 1, 512)

In [67]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

In [74]:
model.compile("adam", loss="binary_crossentropy", metrics=['accuracy']) #You combile the model with the loss and optimiser
model.fit(train_data, train_labels, epochs=10, batch_size=batch_size)

Epoch 1/10
2288/2288 [==============================] - 1s 293us/step - loss: 0.5987 - accuracy: 0.6635
Epoch 2/10
2288/2288 [==============================] - 0s 188us/step - loss: 0.5110 - accuracy: 0.7347
Epoch 3/10
2288/2288 [==============================] - 0s 214us/step - loss: 0.4802 - accuracy: 0.7605
Epoch 4/10
2288/2288 [==============================] - 0s 202us/step - loss: 0.4606 - accuracy: 0.7653
Epoch 5/10
2288/2288 [==============================] - 0s 203us/step - loss: 0.4490 - accuracy: 0.7819
Epoch 6/10
2288/2288 [==============================] - 0s 208us/step - loss: 0.4231 - accuracy: 0.7941
Epoch 7/10
2288/2288 [==============================] - 0s 209us/step - loss: 0.4147 - accuracy: 0.7985
Epoch 8/10
2288/2288 [==============================] - 1s 245us/step - loss: 0.4008 - accuracy: 0.8156
Epoch 9/10
2288/2288 [==============================] - 0s 211us/step - loss: 0.3840 - accuracy: 0.8265
Epoch 10/10
2288/2288 [==============================] - 1s 224u

In [75]:
model.save_weights("top_model.h5")

In order to perform fine-tuning, all layers should start with properly trained weights: for instance you should not slap a randomly initialized fully-connected network on top of a pre-trained convolutional base. This is because the large gradient updates triggered by the randomly initialized weights would wreck the learned weights in the convolutional base. In our case this is why we first train the top-level classifier, and only then start fine-tuning convolutional weights alongside it.

This makes a lot of sense the learning of the top level classifier would initially adversely affect the top level convolution block. This would be as bad training the convolution block from scratch if not worse

##### Fine-tuning should be done with a very slow learning rate, and typically with the SGD optimizer rather than an adaptative learning rate optimizer such as RMSProp. This is to make sure that the magnitude of the updates stays very small, so as not to wreck the previously learned features.

Again, this is great advice for fine tuning. The training is constantly updating the filter weights in the convolution layer. It is immportant that we tread very lighlty with these heavy weights

In [152]:
top_model = Sequential()
top_model.add(Flatten())
top_model.add(Dense(256, activation="relu"))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation="sigmoid"))

In [153]:
top_model

ValueError: This model has not yet been built. Build the model first by calling build() or calling fit() with some data. Or specify input_shape or batch_input_shape in the first layer for automatic build. 

In [154]:
top_model.load_weights("top_model.h5")

ValueError: You are trying to load a weight file containing 2 layers into a model with 0 layers.

In [143]:
new_model = VGG16(weights="imagenet", include_top=False, input_shape=(150,150,3))

In [145]:
seq_vgg = Sequential()
for l in new_model.layers:
    seq_vgg.add(l)

In [155]:
seq_vgg.add(Flatten())

In [156]:
seq_vgg.add(Dense(256, activation="relu"))
seq_vgg.add(Dropout(0.5))
seq_vgg.add(Dense(1, activation="sigmoid"))

In [159]:
new_model